In [154]:
import time
import requests
import numpy as np
import pandas as pd
from scrapy.http import TextResponse
import re

Problem 1


In [155]:
def books_scraper(url,base_url="http://books.toscrape.com/"):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8") 
    prices = response.css("p.price_color::text").extract()
    price = [float(i.replace("Â£", "")) for i in prices]
    book_url = response.css("h3 >a::attr(href)").extract()
    pic_url = response.css("img::attr(src)").extract()
    star = response.css("p[class^='star-rating']::attr(class)").extract()
    star_rating = []
    for i in star:
        star_rating.append(i.replace("star-rating", ""))   
    base_url = "http://books.toscrape.com/catalogue/"
    bookurl = [base_url + i for i in book_url]
    picurl = [base_url + i for i in pic_url]
    genere = []
    desc = []
    for i in bookurl:
        page = requests.get(i)
        response = TextResponse(body=page.text,url=i,encoding="utf-8")
        genere.append(response.css("li~li~li > a::text")[0].extract())
        desc.append(response.css("article[class='product_page'] > p::text").extract_first())
    
    return pd.DataFrame({  "Price":price, "BooksHyperlinks":bookurl, "PicsHyperlinks":picurl,"Star_Ratings":star_rating,'Generes':genere,"AboutBook":desc})

In [156]:
books = []
for i in range(1,100):
    cureent_page =books_scraper(url = f"http://books.toscrape.com/catalogue/page-{i}.html")
    if cureent_page.shape[0] == 0:
        break
    else:
        books.append(cureent_page)

In [157]:
books = pd.concat(books)
books

,Price,BooksHyperlinks,PicsHyperlinks,Star_Ratings,Generes,AboutBook
0,51.77,http://books.toscrape.com/catalogue/a-light-in...,http://books.toscrape.com/catalogue/../media/c...,Three,Poetry,It's hard to imagine a world without A Light i...
1,53.74,http://books.toscrape.com/catalogue/tipping-th...,http://books.toscrape.com/catalogue/../media/c...,One,Historical Fiction,"""Erotic and absorbing...Written with starling ..."
2,50.10,http://books.toscrape.com/catalogue/soumission...,http://books.toscrape.com/catalogue/../media/c...,One,Fiction,"Dans une France assez proche de la nÃ´tre, un ..."
3,47.82,http://books.toscrape.com/catalogue/sharp-obje...,http://books.toscrape.com/catalogue/../media/c...,Four,Mystery,"WICKED above her hipbone, GIRL across her hear..."
4,54.23,http://books.toscrape.com/catalogue/sapiens-a-...,http://books.toscrape.com/catalogue/../media/c...,Five,History,From a renowned historian comes a groundbreaki...
...,...,...,...,...,...,...
15,55.53,http://books.toscrape.com/catalogue/alice-in-w...,http://books.toscrape.com/catalogue/../media/c...,One,Classics,None
16,57.06,http://books.toscrape.com/catalogue/ajin-demi-...,http://books.toscrape.com/catalogue/../media/c...,Four,Sequential Art,High school student Kei Nagai is struck dead i...
17,16.97,http://books.toscrape.com/catalogue/a-spys-dev...,http://books.toscrape.com/catalogue/../media/c...,Five,Historical Fiction,"In Englandâs Regency era, manners and elegan..."
18,53.98,http://books.toscrape.com/catalogue/1st-to-die...,http://books.toscrape.com/catalogue/../media/c...,One,Mystery,"James Patterson, bestselling author of the Ale..."


In [158]:
books.to_csv('books.csv', index=False)

In [159]:
books.to_csv('books.csv', index=False)

In [160]:
books["Price"].max()

59.99

In [161]:
books[books['Price'] == books['Price'].max()]['Generes']

8    Romance
Name: Generes, dtype: object

In [162]:
books['Star_Ratings'].value_counts()

 One      226
 Three    203
 Five     196
 Two      196
 Four     179
Name: Star_Ratings, dtype: int64

In [163]:
rating_map = {
    'Two': 2,
    'One': 1,
    'Five': 5,
    'Four': 4,
    'Three': 3
}

In [164]:
books['Star_Ratings'] = books['Star_Ratings'].apply(lambda x: x.strip())

In [165]:
books['Star_Ratings'] =books['Star_Ratings'].replace(rating_map)

In [166]:
books.corr()

,Price,Star_Ratings
Price,1.000000,0.028166
Star_Ratings,0.028166,1.000000


In [167]:
books.groupby('Star_Ratings').mean()['Price']

Star_Ratings
1    34.561195
2    34.810918
3    34.692020
4    36.093296
5    35.374490
Name: Price, dtype: float64

Problem 2

In [168]:
def jobs_scraper(url):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")
    companies_name = response.xpath("//p[@class='job_list_company_title']/text()").extract()
    vac_name = response.xpath("//p[@class='font_bold']/text()").extract()
    base_url = "https://staff.am"
    urls = response.xpath("//div[@class='web_item_card hs_job_list_item']/a/@href").extract()
    vacs_url = [base_url + i for i in urls]
    deadline1 = response.css("div[class = 'job-inner job-list-deadline'] >p:not([class='job_location'])")
    deadline2 =[i.css('::text').extract()[1] for i in deadline1]
    deadline = [i.replace('\n'," ") for i in deadline2]
    location1 = response.css("div[class = 'job-inner job-list-deadline'] >p[class='job_location']")
    location2 = [i.css("::text").extract()[1] for i in location1]
    location = [i.replace('\n',"").strip() for i in location2]
    return pd.DataFrame({"Companies":companies_name,"Vacancies":vac_name,'Links':vacs_url,"Deadline":deadline,'Location':location})

In [169]:
jobs = []
for i in range(1,100):
    cureent_page =jobs_scraper(url = f"https://staff.am/en/jobs?page={i}&per-page=50")
    if cureent_page.shape[0] == 0:
        break
    else:
        jobs.append(cureent_page)

In [170]:
jobs= pd.concat(jobs)
jobs

,Companies,Vacancies,Links,Deadline,Location
0,Workfront Armenia,Senior Software Engineer,https://staff.am/en/senior-software-engineer-73,31 July 2020,Yerevan
1,Fast Shift LLC,React.js Developer,https://staff.am/en/reactjs-developer-93,31 July 2020,Yerevan
2,Veolia Armenia,HR Specialist,https://staff.am/en/hr-specialist-108,31 July 2020,Yerevan
3,Webb Fontaine Holding LLC,Scrum Master,https://staff.am/en/scrum-master-95,24 July 2020,Yerevan
4,Digitain,Digital Marketing Manager,https://staff.am/en/digital-marketing-manager-39,31 July 2020,Yerevan
...,...,...,...,...,...
3,ServiceTitan,Senior Software Engineer,https://staff.am/en/senior-software-engineer-76,03 July 2020,Yerevan
4,ServiceTitan,"Senior Software Engineer, Front-End",https://staff.am/en/sr-software-engineer-front...,03 July 2020,Yerevan
5,ServiceTitan,Senior Site Reliability Engineer,https://staff.am/en/senior-site-reliability-en...,15 July 2020,Yerevan
6,ServiceTitan,Lead Software Engineer,https://staff.am/en/lead-software-engineer,05 July 2020,Yerevan


In [171]:
jobs['Companies'].value_counts()

Digitain              33
SoftConstruct         23
PicsArt               22
ServiceTitan          15
TeamViewer Armenia    12
                      ..
Medline Clinic LLC     1
BeSofted Group         1
«Teghout» CJSC         1
Մեգա Ֆուդ ՍՊԸ          1
TT soft                1
Name: Companies, Length: 230, dtype: int64

In [172]:
jobs[jobs['Vacancies'].str.contains("Data")]

,Companies,Vacancies,Links,Deadline,Location
26,PicsArt,Data Scientist,https://staff.am/en/data-scientist-82,31 July 2020,Yerevan
27,PicsArt,Data Engineer (Spark),https://staff.am/en/data-engineer-spark-6,31 July 2020,Yerevan
22,buymie,Data analyst,https://staff.am/en/data-analyst-53,24 July 2020,Yerevan
46,Webb Fontaine Holding LLC,Data Engineer,https://staff.am/en/data-engineer-51,22 July 2020,Yerevan
31,ACBA-CREDIT AGRICOLE BANK CJSC,Junior Database Administrator,https://staff.am/en/junior-database-administra...,19 July 2020,Yerevan
4,Ucom CJSC,Senior Database Administrator,https://staff.am/en/senior-database-administra...,11 July 2020,Yerevan
28,Krisp,Data Specialist / QA Engineer,https://staff.am/en/data-specialist-qa-engineer-1,03 July 2020,Yerevan
41,BIVAC Armenia CJSC,Database Administrator,https://staff.am/en/database-administrator-83,02 July 2020,Yerevan
2,DISQO,"Engineering Manager, Java Microservices & Data",https://staff.am/en/engineering-manager-java-m...,03 July 2020,Yerevan


In [173]:
jobs[jobs['Vacancies'].str.contains("Data")]['Vacancies'].count()

9

In [174]:
jobs.to_csv('jobs.csv', index=False)